In [ ]:
import torch
from glob import glob
import sys
import os
import shutil

img_root = r"C:\Users\jiyoo\workspace\MakeAIWork3\project3\apple_disease_classification"

img_folders = [
    "Test/Rot_Apple",
    "Test/Scab_Apple",
    "Test/Blotch_Apple",
    "Test/Normal_Apple",
    "Train/Rot_Apple",
    "Train/Scab_Apple",
    "Train/Blotch_Apple",
    "Train/Normal_Apple",
    
]

In [ ]:
output_folder = os.path.join(img_root, "image_control")
os.makedirs(output_folder, exist_ok=True)  # Create the output folder if it doesn't exist


In [ ]:
sys.path.insert(0, r"C:\Users\jiyoo\workspace\MakeAIWork3\yolov5\models")
from yolov5.models.experimental import attempt_load

In [ ]:
model = attempt_load(weights=r"C:\Users\jiyoo\workspace\MakeAIWork3\yolov5s.pt")

In [ ]:
for img_folder in img_folders:
    img_path = os.path.join(img_root, img_folder)
    img_list = glob(img_path + '/*.jpg')
    img_list.extend(glob(img_path + '/*.png'))

    for img_path in img_list:
        results = model(img_path)
        print(img_path)

        # Apple herkennen met YOLOv5
        apple_results = results.pred[0][results.pred[0][:, -1] == 0]  # Filter resultaten voor appel (klasse 0)

        # Masker toepassen op de appelvorm
        masks = apple_results[:, 4:].argmax(dim=1).unsqueeze(1).float()
        masked_results = torch.cat((apple_results[:, :4], masks), dim=1)

        # Create the corresponding subfolder structure in the output folder
        rel_path = os.path.relpath(img_path, img_root)
        output_subfolder = os.path.join(output_folder, os.path.dirname(rel_path))
        os.makedirs(output_subfolder, exist_ok=True)

        # Save the result to the corresponding location in the output folder
        output_file = os.path.join(output_subfolder, os.path.basename(img_path))
        try:
            masked_results.save(output_file)
            print("Saved result to:", output_file)
        except Exception as e:
            print("Error saving result:", str(e))

        for pred in apple_results:
            tag = results.names[int(pred[-1])]
            print(tag)
